In [38]:
import pandas as pd
import numpy as np
from email import parser
import re 
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('all')

from collections import Counter

df = pd.read_csv('./data/crawled/emails.csv')

# removing root directory crawler values
df = df.iloc[3:]

df

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/ayushgala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nl

,Employee,EmailContent
3,arnold-j,Message-ID: <19171686.1075857585034.JavaMail.e...
4,arnold-j,Message-ID: <29887033.1075857630725.JavaMail.e...
5,arnold-j,Message-ID: <29084893.1075849630138.JavaMail.e...
6,arnold-j,Message-ID: <30248874.1075857584813.JavaMail.e...
7,arnold-j,Message-ID: <2526360.1075857584903.JavaMail.ev...
...,...,...
517549,stf,Message-ID: <18618854.1075840028791.JavaMail.e...
517550,stf,Message-ID: <14350892.1075840028690.JavaMail.e...
517551,stf,Message-ID: <21704474.1075840029683.JavaMail.e...
517552,stf,Message-ID: <9367927.1075840029633.JavaMail.ev...


In [39]:
df.isnull().sum()

Employee          0
EmailContent    150
dtype: int64

In [40]:
df.dropna()

,Employee,EmailContent
3,arnold-j,Message-ID: <19171686.1075857585034.JavaMail.e...
4,arnold-j,Message-ID: <29887033.1075857630725.JavaMail.e...
5,arnold-j,Message-ID: <29084893.1075849630138.JavaMail.e...
6,arnold-j,Message-ID: <30248874.1075857584813.JavaMail.e...
7,arnold-j,Message-ID: <2526360.1075857584903.JavaMail.ev...
...,...,...
517549,stf,Message-ID: <18618854.1075840028791.JavaMail.e...
517550,stf,Message-ID: <14350892.1075840028690.JavaMail.e...
517551,stf,Message-ID: <21704474.1075840029683.JavaMail.e...
517552,stf,Message-ID: <9367927.1075840029633.JavaMail.ev...


In [41]:
df = df.replace(to_replace='None', value=np.nan).dropna()
df = df.astype(str)

# s = pd.Series(df['EmailContent'])
# s = s.convert_dtypes(convert_string=True)

In [42]:
df.dtypes

Employee        object
EmailContent    object
dtype: object

In [43]:
df.isna().sum()

Employee        0
EmailContent    0
dtype: int64

In [44]:
def parse_email(email_content):
    features = {}
    email_parser = parser.Parser()
    
    try:
        email = email_parser.parsestr(email_content)

        # 1. Subject of email
        features['subject'] = email['subject']

        # 2. No. of receivers including CC and BCC
        receivers = []
        for field in ['to', 'cc', 'bcc']:
            if email[field]:
                receivers.extend(re.findall(r'[\w\.-]+@[\w\.-]+', email[field]))
        features['num_receivers'] = len(receivers)

         # 3. Length of email
        body = email.get_payload()
        if email.is_multipart():
            body = ''.join(part.get_payload(decode=True).decode() for part in email.get_payload() if isinstance(part.get_payload(), str))
        features['length_of_email'] = len(body)

        # 4. Email domain name
        from_match = re.search(r'[\w\.-]+@([\w\.-]+)', email['from'])
        features['email_domain'] = from_match.group(1) if from_match else None

        # 6. Sent time
        features['sent_time'] = email['date']

        # 7. Whether the email is forwarded one or not
        features['is_forwarded'] = 1 if (('fwd:' in email['subject'].lower()) or ('fw' in email['subject'].lower())) else 0

        # 8. Sender's email address
        sender_match = re.search(r'([\w\.-]+@[\w\.-]+)', email['from'])
        features['sender_email'] = sender_match.group(1) if sender_match else None

        # # NLP-related semantic features
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(body.lower())
        filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

        # 10. Unique word count (excluding stop words)
        features['email_length'] = len(filtered_words)
    
    except:
        # print(email_content)
        features['subject'] = np.nan
        features['num_receivers'] = np.nan
        features['email_length'] = np.nan
        features['email_domain'] = np.nan
        features['sent_time'] = np.nan
        features['is_forwarded'] = np.nan
        features['sender_email'] = np.nan


    return features

In [45]:
parsed_features = df['EmailContent'].apply(parse_email)

# Create new columns in the dataframe with the extracted features
for feature in ['sender_email', 'subject', 'num_receivers', 'email_length', 'email_domain', 'sent_time', 'is_forwarded']:
    df[feature] = parsed_features.apply(lambda x: x[feature])

df.isna().sum()

Employee         0
EmailContent     0
sender_email     2
subject          1
num_receivers    1
email_length     1
email_domain     2
sent_time        1
is_forwarded     1
dtype: int64

In [ ]:
df = df.dropna(axis=0, subset=['email_length, email_domain'])
df.isna().sum()

Employee         0
EmailContent     0
sender_email     1
subject          0
num_receivers    0
email_length     0
email_domain     1
sent_time        0
is_forwarded     0
dtype: int64

In [48]:
df

,Employee,EmailContent,sender_email,subject,num_receivers,email_length,email_domain,sent_time,is_forwarded
3,arnold-j,Message-ID: <19171686.1075857585034.JavaMail.e...,slafontaine@globalp.com,re:summer inverses,1.0,29.0,globalp.com,"Fri, 8 Dec 2000 05:05:00 -0800 (PST)",0.0
4,arnold-j,Message-ID: <29887033.1075857630725.JavaMail.e...,iceoperations@intcx.com,The WTI Bullet swap contracts,2.0,110.0,intcx.com,"Tue, 15 May 2001 09:43:00 -0700 (PDT)",0.0
5,arnold-j,Message-ID: <29084893.1075849630138.JavaMail.e...,jeff.youngflesh@enron.com,Invitation: EBS/GSS Meeting w/Bristol Babcock ...,4.0,17.0,enron.com,"Mon, 27 Nov 2000 01:49:00 -0800 (PST)",0.0
6,arnold-j,Message-ID: <30248874.1075857584813.JavaMail.e...,caroline.abramo@enron.com,Harvard Mgmt,3.0,53.0,enron.com,"Tue, 12 Dec 2000 09:33:00 -0800 (PST)",0.0
7,arnold-j,Message-ID: <2526360.1075857584903.JavaMail.ev...,klarnold@flash.net,Fwd: NYTimes.com Article: Suspended Rabbi Quit...,1.0,467.0,flash.net,"Mon, 11 Dec 2000 23:52:00 -0800 (PST)",1.0
...,...,...,...,...,...,...,...,...,...
517549,stf,Message-ID: <18618854.1075840028791.JavaMail.e...,jodi.droll@xcelenergy.com,RE: Scheduling Time constant,1.0,259.0,xcelenergy.com,"Thu, 11 Oct 2001 09:39:47 -0700 (PDT)",0.0
517550,stf,Message-ID: <14350892.1075840028690.JavaMail.e...,mark.hackney@aps.com,"FW: Standards Announcement - October 17, 2001",1.0,124.0,aps.com,"Wed, 17 Oct 2001 16:11:39 -0700 (PDT)",1.0
517551,stf,Message-ID: <21704474.1075840029683.JavaMail.e...,paul.rice@pacificorp.com,Late tags,1.0,20.0,pacificorp.com,"Fri, 6 Apr 2001 01:42:00 -0700 (PDT)",0.0
517552,stf,Message-ID: <9367927.1075840029633.JavaMail.ev...,gjcarter@bpa.gov,RE: BCHA Automatic Denial/Approval,18.0,167.0,bpa.gov,"Mon, 9 Apr 2001 23:11:00 -0700 (PDT)",0.0


In [50]:
# Save the updated dataframe to a new CSV file
df.to_csv('./data/parsed/parsed_emails.csv', index=False)